# 辽宁高速公路裂缝检测

In [1]:
%%time
import sagemaker
from sagemaker import get_execution_role
 
role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::002224604296:role/service-role/AmazonSageMaker-ExecutionRole-20200402T124851
CPU times: user 834 ms, sys: 182 ms, total: 1.02 s
Wall time: 10.4 s


# S3路径

In [2]:
bucket = sess.default_bucket()  
prefix = 'semantic-segmentation-demo'
print(bucket)

sagemaker-us-east-1-002224604296


# 准备训练镜像

In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


811284229777.dkr.ecr.us-east-1.amazonaws.com/semantic-segmentation:1


# 准备数据

In [4]:
import glob
num_training_samples=len(glob.glob1('train',"*.jpg"))

print ( ' Num Train Images = ' + str(num_training_samples))
assert num_training_samples == len(glob.glob1('train_annotation',"*.png"))

print ( ' Num Validation Images = ' + str(len(glob.glob1('validation',"*.jpg"))))
assert len(glob.glob1('validation',"*.jpg")) == len(glob.glob1('validation_annotation',"*.png"))

 Num Train Images = 170
 Num Validation Images = 31


In [5]:
# Create channel names for the s3 bucket.
train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'
# test_input_channel = prefix + '/test_input_20200511'
test_input_channel = prefix + '/test_input_liaoning_sample'
test_output_channel = prefix + '/test_output'
# label_map_channel = prefix + '/label_map'

# 上传数据到S3

In [6]:
%%time
# upload the appropraite directory up to s3 respectively for all directories.
sess.upload_data(path='train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='validation', bucket=bucket, key_prefix=validation_channel)
sess.upload_data(path='train_annotation', bucket=bucket, key_prefix=train_annotation_channel)
sess.upload_data(path='validation_annotation', bucket=bucket, key_prefix=validation_annotation_channel)
# sess.upload_data(path='train_label_map.json', bucket=bucket, key_prefix=label_map_channel)

CPU times: user 3.3 s, sys: 354 ms, total: 3.65 s
Wall time: 1min 5s


's3://sagemaker-us-east-1-002224604296/semantic-segmentation-demo/validation_annotation'

In [7]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
print(s3_output_location)

s3://sagemaker-us-east-1-002224604296/semantic-segmentation-demo/output


# 准备训练

In [13]:
# Create the sagemaker estimator object.
ss_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count = 1, 
                                         train_instance_type = 'ml.p3.2xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         output_path = s3_output_location,
                                         base_job_name = 'ss-notebook-demo',
                                         sagemaker_session = sess)

## 设置训练参数

In [14]:
# Setup hyperparameters 
ss_model.set_hyperparameters(backbone='resnet-101', # This is the encoder. Other option is resnet-50
                             algorithm='deeplab', # This is the decoder. Other option is 'psp' and 'deeplab'                             
                             use_pretrained_model='True', # Use the pre-trained model.
                             crop_size=240, # Size of image random crop.                             
                             num_classes=2, # Pascal has 21 classes. This is a mandatory parameter.
                             epochs=20, # Number of epochs to run.
                             learning_rate=0.0001,                             
                             optimizer='adam', # Other options include 'adam', 'rmsprop', 'nag', 'adagrad'.
                             lr_scheduler='poly', # Other options include 'cosine' and 'step'.                           
                             mini_batch_size=16, # Setup some mini batch size.
                             validation_mini_batch_size=16,
                             early_stopping=True, # Turn on early stopping. If OFF, other early stopping parameters are ignored.
                             early_stopping_patience=2, # Tolerate these many epochs if the mIoU doens't increase.
                             early_stopping_min_epochs=10, # No matter what, run these many number of epochs.                             
                             num_training_samples=num_training_samples) # This is a mandatory parameter, 1464 in this case.

In [15]:
# Create full bucket names
s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucket, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucket, validation_annotation_channel)

distribution = 'FullyReplicated'
# Create sagemaker s3_input objects
train_data = sagemaker.session.s3_input(s3_train_data, distribution=distribution, 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution=distribution, 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')
train_annotation = sagemaker.session.s3_input(s3_train_annotation, distribution=distribution, 
                                        content_type='image/png', s3_data_type='S3Prefix')
validation_annotation = sagemaker.session.s3_input(s3_validation_annotation, distribution=distribution, 
                                        content_type='image/png', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 
                 'validation': validation_data,
                 'train_annotation': train_annotation, 
                 'validation_annotation':validation_annotation}

## 训练模型

In [16]:
ss_model.fit(inputs=data_channels, logs=True)

2020-04-29 01:48:57 Starting - Starting the training job...
2020-04-29 01:48:58 Starting - Launching requested ML instances......
2020-04-29 01:50:07 Starting - Preparing the instances for training......
2020-04-29 01:51:20 Downloading - Downloading input data
2020-04-29 01:51:20 Training - Downloading the training image........Docker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
[04/29/2020 01:52:39 INFO 140228826310464] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'syncbn': u'False', u'gamma2': u'0.9', u'gamma1': u'0.9', u'early_stopping_min_epochs': u'5', u'epochs': u'10', u'_workers': u'16', u'_num_kv_servers': u'auto', u'weight_decay': u'0.0001', u'crop_size': u'240', u'use_pretrained_model': u'True', u'_aux_weight': u'0.5', u'_hybrid': u'False', u'_augmentation_type': u'default', u'lr_scheduler': u'poly', u'early_stopping_patie

[04/29/2020 01:53:37 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 2, train loss: 0.10858195945620537 .
[04/29/2020 01:53:37 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 2, train throughput: 17.5463143867 samples/sec.
[04/29/2020 01:53:39 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 2, validation pixel_accuracy: 0.9598350694444444 .
[04/29/2020 01:53:39 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 2, validation mIOU: 0.7040224326012159 .
[04/29/2020 01:53:39 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 2, validation throughput: 61.2093255733 samples/sec.
[04/29/2020 01:53:39 INFO 140228826310464] Serializing model to /opt/ml/model/model_best.params
[04/29/2020 01:53:39 INFO 140228826310464] #progress_metric: host=algo-1, completed 15 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 0, "sum": 0.0, "min": 0}, "Number of Batches Since Last Reset": {"count": 1, "max": 0, 

[04/29/2020 01:55:03 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 7, train loss: 0.06036086075685241 .
[04/29/2020 01:55:03 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 7, train throughput: 17.4202879077 samples/sec.
[04/29/2020 01:55:06 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 7, validation pixel_accuracy: 0.9827164713541666 .
[04/29/2020 01:55:06 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 7, validation mIOU: 0.8454581702587733 .
[04/29/2020 01:55:06 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 7, validation throughput: 61.4904507681 samples/sec.
[04/29/2020 01:55:06 INFO 140228826310464] Serializing model to /opt/ml/model/model_best.params
[04/29/2020 01:55:06 INFO 140228826310464] #progress_metric: host=algo-1, completed 40 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 0, "sum": 0.0, "min": 0}, "Number of Batches Since Last Reset": {"count": 1, "max": 0, 

[04/29/2020 01:56:46 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 13, train loss: 0.05154777355492115 .
[04/29/2020 01:56:46 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 13, train throughput: 17.5036904826 samples/sec.
[04/29/2020 01:56:49 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 13, validation pixel_accuracy: 0.9843614366319444 .
[04/29/2020 01:56:49 INFO 140228826310464] #quality_metric. host: algo-1, epoch: 13, validation mIOU: 0.859218594864106 .
[04/29/2020 01:56:49 INFO 140228826310464] #throughput_metric. host: algo-1, epoch: 13, validation throughput: 60.6975625595 samples/sec.
[04/29/2020 01:56:49 INFO 140228826310464] #progress_metric: host=algo-1, completed 70 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 0, "sum": 0.0, "min": 0}, "Number of Batches Since Last Reset": {"count": 1, "max": 0, "sum": 0.0, "min": 0}, "Number of Records Since Last Reset": {"count": 1, "max": 0, "sum": 0

# 部署与验证

In [17]:
ss_predictor = ss_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

-----------------!

In [8]:
import boto3
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

In [9]:
# import glob
# import os
# import shutil
# import PIL
# from PIL import Image
# import numpy as np
# import io
# import matplotlib.pyplot as plt

with open(test_filename, 'rb') as image:
    img = image.read()
    img = bytearray(img)

response = runtime_sm_client.invoke_endpoint(
                        EndpointName='psp-seg-exp-aoyu-20200427-10-51-49-837',
                        ContentType='image/jpeg',
                        Accept='image/png',
                        Body=img)
# mask1 = np.array(Image.open(io.BytesIO(response['Body'].read())))
# num_classes = 2
# mask1 = np.fliplr(np.transpose(mask1))
# plt.imshow(mask1, vmin=0, vmax=num_classes-1, cmap='jet')
# plt.show()

NameError: name 'test_filename' is not defined

# 优化点：
## 后处理步骤，增强分割的效果
## 批处理方式，方便批量处理数据

In [36]:
#prepare data for patch transform
# prepare data
import matplotlib.pyplot as plt
import PIL
import glob
from os import path as osp

raw_path = 'raw'
save_path = 'batch_test'
height = 1520
width = 1024

for f in glob.glob1(raw_path, "*.jpg"):
    im = PIL.Image.open(osp.join(raw_path,f))
    im.thumbnail([height,width],PIL.Image.ANTIALIAS)
    im.save(osp.join(save_path,f), "JPEG")
    im2 = im.rotate(90, expand=True)
    f2 = f.split('.')[0]+'_r.jpg'
    im2.save(osp.join(save_path,f2), "JPEG")

In [10]:
import glob
import os
import shutil
import PIL
from PIL import Image
import numpy as np
import io
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import boto3
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')


def liaoning_test(dirr):
    prefix = '.'
    train_prefix = os.path.join(prefix,'train')
    val_prefix = os.path.join(prefix,'validation')
    test_prefix = os.path.join(prefix,'test')
    dir_name_output = 'batch_test_result_instance'
    
    os.makedirs(train_prefix, exist_ok=True)
    os.makedirs(val_prefix, exist_ok=True)
    os.makedirs(test_prefix, exist_ok=True)
    # dir_list = ['train','validation','test']
    dir_list = ['batch_test']
    for dirr in dir_list:
        for filename in glob.glob1(dirr,"*.jpg"):
            %%time
            if filename.split('.')[0][-2:] == '_r':
                continue
            #         test_filename = filename.split('.')[0]+'_r.jpg'
            #         test_filename = Path(os.path.join(dirr,test_filename))
            #         if test_filename.exists():
            #             continue
            # resize image size for inference
            filename = os.path.join(dirr,filename)
            im1 = PIL.Image.open(filename)
            im1.thumbnail([800,600],PIL.Image.ANTIALIAS)
            #         im1.thumbnail([1520,1024],PIL.Image.ANTIALIAS)
            im1.save(filename, "JPEG")
            ff = filename.split('/')[1]
            save_file = os.path.join(prefix,dirr,ff)
            #         print(save_file)
            with open(filename, 'rb') as image:
                img = image.read()
                img = bytearray(img)

            #         ss_predictor.content_type = 'image/jpeg'
            #         ss_predictor.accept = 'image/png'
            #         results = ss_predictor.predict(img)
            response = runtime_sm_client.invoke_endpoint(
                            EndpointName='ss-notebook-demo-2020-04-29-01-48-56-769',
                            ContentType='image/jpeg',
                            Accept='image/png',
                            Body=img)
            mask = np.array(Image.open(io.BytesIO(response['Body'].read())))
            num_classes = 2
            #         mask = np.array(Image.open(io.BytesIO(results)))
            #         plt.imshow(mask, vmin=0, vmax=num_classes-1, cmap='jet')
            #         plt.savefig(save_file, bbox_inches='tight')
            #plt.show()
            if dirr == 'batch_test':
                im2 = PIL.Image.open(filename)
                im2 = im2.rotate(90, expand=True)
                im2.thumbnail([600,800],PIL.Image.ANTIALIAS)
            #             im2.thumbnail([1024,1520],PIL.Image.ANTIALIAS)
                rfilename = filename.split('.')[0]+'_r.jpg'
                im2.save(rfilename, "JPEG")
                with open(rfilename, 'rb') as image:
                    img = image.read()
                    img = bytearray(img)
            #             rresults = ss_predictor.predict(img)
                response = runtime_sm_client.invoke_endpoint(
                            EndpointName='ss-notebook-demo-2020-04-29-01-48-56-769',
                            ContentType='image/jpeg',
                            Accept='image/png',
                            Body=img)
                rmask = np.array(Image.open(io.BytesIO(response['Body'].read()))) 
            #             rmask = np.array(Image.open(io.BytesIO(rresults)))
            #             rsave_file = os.path.join(prefix,dirr,ff.split('.')[0]+"_r.jpg")
            #             plt.imshow(rmask, vmin=0, vmax=num_classes-1, cmap='jet')
            #             plt.savefig(rsave_file, bbox_inches='tight')
                rmask = np.fliplr(np.transpose(rmask))|mask
                ret, binary = cv2.threshold(rmask*255,127,255,cv2.THRESH_BINARY)

            #             print(dir_name_raw)
                contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                raw_img = cv2.imread(filename)
                ffresult = raw_img
                contours_poly = [None]*len(contours)
                boundRect = [None]*len(contours)
                for i, c in enumerate(contours):
                    if c.size < contour_size_limit:
                        continue
                contours_poly[i] = cv2.approxPolyDP(c, 3, True)
                boundRect[i] = cv2.boundingRect(contours_poly[i])
                cv2.rectangle(ffresult, (int(boundRect[i][0]), int(boundRect[i][1])), \
                              (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
                #fill contours
                colormask = np.zeros(ffresult.shape,np.uint8)
                cv2.fillPoly(colormask, pts = [c], color = color)
                ffresult = cv2.addWeighted(ffresult, 1.0, colormask, 0.2, 0)
                cv2.imwrite(osp.join(dir_name_output,filename.split('/')[-1]), ffresult)
#                 plt.imshow(ffresult)
                print("finish process file {} !!!!!!!!!".format(filename.split('/')[-1]))
    return 1



#             fmask = np.fliplr(np.transpose(rmask))|mask
# #             ret, binary = cv2.threshold(fmask*255,127,255,cv2.THRESH_BINARY)
# #             contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#             raw_img = cv2.imread(filename)
# #             cv2.drawContours(raw_img,contours,-1,(0,0,255),3)
#             fsave_file = os.path.join(prefix,dirr,ff.split('.')[0]+"_f.jpg")
# #             cv2.imwrite(fsave_file, raw_img)
#             colorfmask = np.zeros([fmask.shape[0],fmask.shape[1],3],np.uint8)
#             colorfmask[:,:,0] = np.zeros(fmask.shape)
#             colorfmask[:,:,1] = fmask*255
#             colorfmask[:,:,2] = np.zeros(fmask.shape)
#             ffresult = cv2.addWeighted(raw_img, 1.0, colorfmask, 0.01, 1.0)
#             cv2.imwrite(fsave_file, ffresult)
#             plt.imshow(fmask, vmin=0, vmax=num_classes-1, cmap='jet')
#             plt.savefig(fsave_file, bbox_inches='tight')
#print ( ' Num Train Images = ' + str(num_training_samples))
#assert num_training_samples == len(glob.glob1('train_annotation',"*.png"))

#print ( ' Num Validation Images = ' + str(len(glob.glob1('validation',"*.jpg"))))
#assert len(glob.glob1('validation',"*.jpg")) == len(glob.glob1('validation_annotation',"*.png"))

# 简化脚本，方便客户评估

In [11]:
liaoning_test(dirr = "./raw")

UsageError: Line magic function `%%time` not found.
